In [1]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
files_in_directory =  os.listdir()
#files_in_directory

In [3]:
file = pd.ExcelFile(files_in_directory[6])

OSError: File contains no valid workbook part

In [ ]:
#file.sheet_names

In [ ]:
df = file.parse('Dataset', usecols= 'A:M')

In [ ]:
df.tail()

#### creating integer columns for 'Quality Score' factors ratings 

In [ ]:
df['Ad relevance'].unique()

In [ ]:
df['Ad relevance score'] = df['Ad relevance'].map({'Below average': 0, 'Above average': 2, 'Average': 1})

In [ ]:
df['Expected click-through rate'].unique()

In [ ]:
df['Expected CTR score'] = df['Expected click-through rate'].map({'Below average': 0, 'Above average': 3.5, 'Average': 1.75})

In [ ]:
df['Landing page experience'].unique()

In [ ]:
df['Landing page exp score'] = df['Landing page experience'].map({'Average': 1.75, 'Above average': 3.5, 'Below average': 0})

In [ ]:
df['Clicks'] = round(df['Clicks'])
df['Clicks'] = df['Clicks'].astype(int)
df['Conversions'] = round(df['Conversions'])
df['Conversions'] = df['Conversions'].astype(int)
df['Total Cost'] = round(df['Total Cost'], 2)
df['Total Revenue'] = round(df['Total Revenue'], 2)

In [ ]:
df.head(3)

### duplicate bids on same keyword

In [ ]:
df[df.duplicated(subset= 'Search Keyword', keep= False)]

In [ ]:
df.head(2)

### Search Keywords similarity

In [ ]:
import numpy as np
from fuzzywuzzy import process, fuzz

In [ ]:
process.extract('posting christmas stocking gift bag large', list(df['Search Keyword']), scorer=fuzz.token_sort_ratio)

In [ ]:
def high_similarity_keywords():
    unique_search_words =  df['Search Keyword'].unique()
    matches_list = []
    for keyword in unique_search_words:
        for match_result in process.extract(keyword, unique_search_words, scorer=fuzz.token_sort_ratio):
            score_tuple =  (keyword,) + match_result
            matches_list.append(score_tuple)

    similarity_df = pd.DataFrame(matches_list, columns=['keyword','match_keyword','score'])
    similarity_df['keyword_compared'] = np.minimum(similarity_df['keyword'], similarity_df['match_keyword'])

    high_score_df = similarity_df[(similarity_df['score'] >= 75) &
                (similarity_df['keyword'] !=  similarity_df['match_keyword']) &
                (similarity_df['keyword_compared'] != similarity_df['match_keyword'])]
    
    high_score_df.drop('keyword_compared',axis=1, inplace = True)
    
    high_similarity_df = high_score_df.groupby(['keyword']).agg(
                        {'score': np.mean,'match_keyword': ",  ".join}).sort_values(
                        ['keyword'], ascending=False).reset_index()
    
    high_similarity_df['score'] = np.round(high_similarity_df['score'])
    high_similarity_df['score'] = high_similarity_df['score'].astype(int)
    return high_similarity_df

In [ ]:
similar_keywords_df =  high_similarity_keywords()

In [ ]:
similar_keywords_df.head()

In [ ]:
matches_df = similar_keywords_df['match_keyword'].str.split(',', expand = True)

In [ ]:
redundant_keywords_dfs =  pd.concat([similar_keywords_df[['keyword', 'score']], matches_df], axis = 1)

In [ ]:
redundant_keywords_dfs.columns = ['keyword', 'score', 'match1', 'match2', 'match3', 'match4']

In [ ]:
redundant_keywords_dfs.head(10)

#### data to be stored in database

In [ ]:
df = df.reset_index()

In [ ]:
df.head(3)

In [ ]:
#df.to_csv('goshc modeling data.csv', index = False)

### Connection to cloud database

In [ ]:
import pypyodbc as podbc

In [ ]:
# conn2 = podbc.connect('Driver={SQL Server};'
#                                 'Server=xxxxxx.cahwy5tcsa.us-east-1.rds.amazonaws.com,xxxx;'
#                                 'Database=Charity;'
#                                 'uid=xxxxx;pwd=xxxxxxx')
# print ("Connected")
# cursor = conn2.cursor()

#### Create 'campaign data' table for data in database

In [ ]:
# create_table = '''
# CREATE TABLE campaign_data (
#     serial_no INT NOT NULL,
#     Ad_Group VARCHAR(100) NOT NULL,
#     Search_Keyword VARCHAR(MAX) NOT NULL,
#     Quality_Score INT NOT NULL,
#     Ad_relevance_Score INT,
#     Expected_CTR_Score DECIMAL(3,2),
#     Landing_page_exp_Score DECIMAL(3,2),
#     Ad_relevance VARCHAR(16) NOT NULL,
#     Expected_CTR VARCHAR(16) NOT NULL,
#     Landing_page_experience VARCHAR(16) NOT NULL,
#     Avg_position INT ,
#     Impressions INT,
#     Clicks INT,
#     Total_Cost DECIMAL(8,2),
#     Conversions INT,
#     Revenue_per_unit_sold DECIMAL(8,2),
#     Total_Revenue DECIMAL(8,2)
# )
# '''

In [ ]:
# cursor.execute(create_table)

In [ ]:
# conn2.commit()

In [ ]:
df.shape

#### function for inserting data into db table

In [ ]:
def insert_data_to_table(serial_no, Ad_Group, Search_Keyword, Quality_Score, Ad_relevance_Score,
                        Expected_CTR_Score, Landing_page_exp_Score, Ad_relevance, Expected_CTR, Landing_page_experience,
                        Avg_position, Impressions, Clicks, Total_Cost,  Conversions, Revenue_per_unit_sold, Total_Revenue):
    query = '''
            INSERT INTO campaign_data(serial_no, Ad_Group, Search_Keyword, Quality_Score, Ad_relevance_Score,
                             Expected_CTR_Score, Landing_page_exp_Score, Ad_relevance, Expected_CTR, Landing_page_experience,
                             Avg_position, Impressions, Clicks, Total_Cost,  Conversions, Revenue_per_unit_sold, Total_Revenue
                             )
              VALUES 
              (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
            '''

    row_values_to_insert = (serial_no, Ad_Group, Search_Keyword, Quality_Score, Ad_relevance_Score,
                        Expected_CTR_Score, Landing_page_exp_Score, Ad_relevance, Expected_CTR, Landing_page_experience,
                        Avg_position, Impressions, Clicks, Total_Cost,  Conversions, Revenue_per_unit_sold, Total_Revenue)
    
    cursor.execute(query, row_values_to_insert)

#### function for using above function

In [ ]:
def add_data(df):
    for i, row in df.iterrows():
        insert_data_to_table(row['index'], row['Ad Group'], row['Search Keyword'], row['Quality score'],
                        row['Ad relevance score'], row['Expected CTR score'], row['Landing page exp score'],
                        row['Ad relevance'], row['Expected click-through rate'], row['Landing page experience'],
                        row['Avg. position'], row['Impressions '], row['Clicks'], row['Total Cost'], row['Conversions'],
                         row['Revenue per unit sold'], row['Total Revenue'] )

In [ ]:
#add_data(df)

In [ ]:
#conn2.commit()

In [ ]:
#conn2.close()